In [1]:
import os
os.chdir('..')

In [2]:
from datgan.datgan import DATGAN

import numpy as np
import pandas as pd
import networkx as nx

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Load the original data

In [3]:
df = pd.read_csv('example/data/CMAP.csv', index_col=False)

In [4]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,3.93477,2,3,3,detached,6,0,30,1,4,PTE,20.166667
1,drive,2,SHOPPING,0.31557,3,3,3,detached,7,0,54,1,5,FTE,17.500000
2,drive,2,SHOPPING,0.28349,1,1,0,detached,3,0,80,1,3,PTE,9.333333
3,drive,2,OTHER,0.69417,2,2,0,detached,5,1,42,1,5,FTE,13.783333
4,passenger,1,SHOPPING,4.30666,2,2,1,detached,4,0,32,0,3,Unemployed,11.566667


# DAG

We need to define the DAG using the library `networkx` and the continuous columns.

In [5]:
continuous_columns = ["distance", "age", "departure_time"]

In [6]:
graph = nx.DiGraph()
graph.add_edges_from([
    ("age", "license"),
    ("age", "education_level"),
    ("gender", "work_status"),
    ("education_level", "work_status"),
    ("education_level", "hh_income"),
    ("work_status", "hh_income"),
    ("hh_income", "hh_descr"),
    ("hh_income", "hh_size"),
    ("hh_size", "hh_vehicles"),
    ("hh_size", "hh_bikes"),
    ("work_status", "trip_purpose"),
    ("trip_purpose", "departure_time"),
    ("trip_purpose", "distance"),
    ("travel_dow", "choice"),
    ("distance", "choice"),
    ("departure_time", "choice"),
    ("hh_vehicles", "choice"),
    ("hh_bikes", "choice"),
    ("license", "choice"),
    ("education_level", "hh_size"),
    ("work_status", "hh_descr"),
    ("work_status", "hh_size"),
    ("hh_income", "hh_bikes"),
    ("hh_income", "hh_vehicles"),
    ("trip_purpose", "choice")
])

# Training the DATGAN

In [7]:
output_folder = './example/output/'

In [8]:
datgan = DATGAN(output=output_folder, max_epoch=1000)

It is possible to preprocess the data and save it somewhere. Since it takes a bit of time to do it, it helps to test multiple models faster. 

In [9]:
datgan.preprocess(df, continuous_columns, preprocessed_data_path='./example/encoded_data')

[0112 12:13:33 @datgan.py:204] Preprocessing the data!
[0112 12:13:33 @data.py:113] Encoding categorical variable "choice"...
[0112 12:13:33 @data.py:113] Encoding categorical variable "travel_dow"...
[0112 12:13:33 @data.py:113] Encoding categorical variable "trip_purpose"...
[0112 12:13:33 @data.py:100] Encoding continuous variable "distance"...
[0112 12:13:33 @data.py:287]   Fitting model with 10 components
[0112 12:13:41 @data.py:314]   Predictions were done on 7 components => Fit with 7 components!
[0112 12:13:48 @data.py:314]   Predictions were done on 6 components => Fit with 6 components!
[0112 12:13:53 @data.py:319]   Predictions were done on 6 components => FINISHED!
[0112 12:13:53 @data.py:322]   Train VGM with full data
[0112 12:13:59 @data.py:113] Encoding categorical variable "hh_vehicles"...
[0112 12:13:59 @data.py:113] Encoding categorical variable "hh_size"...
[0112 12:14:00 @data.py:113] Encoding categorical variable "hh_bikes"...
[0112 12:14:00 @data.py:113] Encoding

If the data has been preprocessed, you need to provide the path where it was saved. The model will then the preprocessed data and work with it.

In [10]:
datgan.fit(df, graph, continuous_columns, preprocessed_data_path='./example/encoded_data')

[0112 12:14:39 @datgan.py:202] Preprocessed data have been loaded!



[0112 12:14:39 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
[0112 12:14:39 @DATGANSynthesizer.py:138] Creating cell for age (in-edges: 0; ancestors: 0)
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0112 12:14:39 @registry.py:126] gen/LSTM/age/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[0112 12:14:39 @registry.py:134] gen/LSTM/age/FC output: [500, 50]
[0112 12:14:39 @registry.py:126] gen/LSTM/age/FC_val input: [500, 50]
[0112 12:14:39 @registry.py:134] gen/LSTM/age/FC_val output: [500, 5]
[0112 12:14:39 @registry.py:126] gen/LSTM/age/FC_prob inp

[0112 12:14:39 @registry.py:134] gen/LSTM/age-gender/FC_noise output: [None, 200]
[0112 12:14:39 @registry.py:126] gen/LSTM/work_status/FC input: [500, 100]
[0112 12:14:39 @registry.py:134] gen/LSTM/work_status/FC output: [500, 50]
[0112 12:14:39 @registry.py:126] gen/LSTM/work_status/FC_prob input: [500, 50]
[0112 12:14:39 @registry.py:134] gen/LSTM/work_status/FC_prob output: [500, 8]
[0112 12:14:39 @registry.py:126] gen/LSTM/work_status/FC_input input: [500, 8]
[0112 12:14:39 @registry.py:134] gen/LSTM/work_status/FC_input output: [500, 100]
[0112 12:14:39 @DATGANSynthesizer.py:138] Creating cell for hh_income (in-edges: 2; ancestors: 4)
[0112 12:14:39 @registry.py:126] gen/LSTM/concat-hh_income/FC_inputs input: [500, 200]
[0112 12:14:39 @registry.py:134] gen/LSTM/concat-hh_income/FC_inputs output: [500, 100]
[0112 12:14:39 @registry.py:126] gen/LSTM/concat-hh_income/FC_states input: [500, 200]
[0112 12:14:39 @registry.py:134] gen/LSTM/concat-hh_income/FC_states output: [500, 100]
[

[0112 12:14:40 @registry.py:126] gen/LSTM/concat-hh_bikes/FC_h_outputs input: [500, 200]
[0112 12:14:40 @registry.py:134] gen/LSTM/concat-hh_bikes/FC_h_outputs output: [500, 100]
[0112 12:14:40 @registry.py:126] gen/LSTM/hh_bikes/FC input: [500, 100]
[0112 12:14:40 @registry.py:134] gen/LSTM/hh_bikes/FC output: [500, 50]
[0112 12:14:40 @registry.py:126] gen/LSTM/hh_bikes/FC_prob input: [500, 50]
[0112 12:14:40 @registry.py:134] gen/LSTM/hh_bikes/FC_prob output: [500, 8]
[0112 12:14:40 @registry.py:126] gen/LSTM/hh_bikes/FC_input input: [500, 8]
[0112 12:14:40 @registry.py:134] gen/LSTM/hh_bikes/FC_input output: [500, 100]
[0112 12:14:40 @DATGANSynthesizer.py:138] Creating cell for choice (in-edges: 7; ancestors: 13)
[0112 12:14:40 @registry.py:126] gen/LSTM/concat-choice/FC_inputs input: [500, 700]
[0112 12:14:40 @registry.py:134] gen/LSTM/concat-choice/FC_inputs output: [500, 100]
[0112 12:14:40 @registry.py:126] gen/LSTM/concat-choice/FC_states input: [500, 700]
[0112 12:14:40 @regis

[0112 12:14:43 @base.py:209] Setup callbacks graph ...







[0112 12:14:43 @utils.py:26] Clip discrim/DISCR_FC_0/FC/W

[0112 12:14:43 @utils.py:26] Clip discrim/DISCR_FC_0/FC/b
[0112 12:14:43 @utils.py:26] Clip discrim/DISCR_FC_0/FC_DIVERSITY/W
[0112 12:14:43 @utils.py:26] Clip discrim/DISCR_FC_0/FC_DIVERSITY/b
[0112 12:14:43 @utils.py:26] Clip discrim/DISCR_FC_0/BN/beta
[0112 12:14:43 @utils.py:26] Clip discrim/DISCR_FC_TOP/W
[0112 12:14:43 @utils.py:26] Clip discrim/DISCR_FC_TOP/b
[0112 12:14:43 @summary.py:46] [MovingAverageSummary] 3 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[0112 12:14:43 @summary.py:93] Summarizing collection 'summaries' of size 4.

[0112 12:14:43 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.

[0112 12:14:44 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[0112 12:14:46 @base.py:236] Initializing the session ...
[0112 12:14:46 

 18%|###############1                                                                      |3/17[00:02<00:10, 1.32it/s]

...

[0112 12:25:12 @base.py:285] Epoch 999 (global_step 25474) finished, time:0.468 second.
[0112 12:25:12 @saver.py:79] Model saved to ./example/output/model\model-25474.


[0112 12:25:12 @monitor.py:467] GAN_loss/d_loss: -0.013703
[0112 12:25:12 @monitor.py:467] GAN_loss/g_loss: -0.12386
[0112 12:25:12 @monitor.py:467] GAN_loss/kl_div: 0.046754
[0112 12:25:12 @monitor.py:467] QueueInput/queue_size: 50
[0112 12:25:12 @base.py:275] Start Epoch 1000 ...


100%|#####################################################################################|17/17[00:00<00:00,37.66it/s]

[0112 12:25:13 @base.py:285] Epoch 1000 (global_step 25499) finished, time:0.452 second.
[0112 12:25:13 @saver.py:79] Model saved to ./example/output/model\model-25499.
[0112 12:25:13 @monitor.py:467] GAN_loss/d_loss: -0.0145
[0112 12:25:13 @monitor.py:467] GAN_loss/g_loss: -0.1229
[0112 12:25:13 @monitor.py:467] GAN_loss/kl_div: 0.045495
[0112 12:25:13 @monitor.py:467] QueueInput/queue_size: 50
[0112 12:25:13 @base.py:289] Training has finished!



[0112 12:25:13 @DATGANSynthesizer.py:138] Creating cell for age (in-edges: 0; ancestors: 0)
[0112 12:25:13 @DATGANSynthesizer.py:138] Creating cell for gender (in-edges: 0; ancestors: 0)
[0112 12:25:13 @DATGANSynthesizer.py:138] Creating cell for travel_dow (in-edges: 0; ancestors: 0)
[0112 12:25:13 @DATGANSynthesizer.py:138] Creating cell for license (in-edges: 1; ancestors: 1)
[0112 12:25:13 @DATGANSynthesizer.py:138] Creating cell for education_level (in-edges: 1; ancestors: 1)
[0112 12:25:13 @DATGANSynthesizer.py:138] Creating cell for work_status (in-edges: 2; ancestors: 3)
[0112 12:25:13 @DATGANSynthesizer.py:138] Creating cell for hh_income (in-edges: 2; ancestors: 4)
[0112 12:25:13 @DATGANSynthesizer.py:138] Creating cell for trip_purpose (in-edges: 1; ancestors: 4)
[0112 12:25:13 @DATGANSynthesizer.py:138] Creating cell for hh_descr (in-edges: 2; ancestors: 5)
[0112 12:25:14 @DATGANSynthesizer.py:138] Creating cell for hh_size (in-edges: 3; ancestors: 5)
[0112 12:25:14 @DATGA

You can save the model to reuse is later.

In [11]:
datgan.save('trained', force=True)

[0112 12:25:32 @datgan.py:519] Model saved successfully.


Sample the synthetic data

In [12]:
samples = datgan.sample(len(df))

|                                                                                          |17/?[00:00<00:00,38.23it/s]


If there is a discrete distribution, you need to round the values. Future work will fix this.

In [13]:
samples.age = np.round(samples.age)

In [14]:
samples.to_csv('example/data/CMAP_synthetic.csv', index=False)